In [15]:
import cv2
import numpy as np

def hMaxCallback(pos):
    pass
def hMinCallback(pos):
    pass
def sMaxCallback(pos):
    pass
def sMinCallback(pos):
    pass
def vMaxCallback(pos):
    pass
def vMinCallback(pos):
    pass

cv2.namedWindow('Test')
cv2.createTrackbar('HueMin', 'Test', 30, 179, hMinCallback)
cv2.createTrackbar('HueMax', 'Test', 150, 179, hMaxCallback) 
cv2.createTrackbar('SatMin', 'Test', 10, 255, sMinCallback) 
cv2.createTrackbar('SatMax', 'Test', 120, 255, sMaxCallback) 
cv2.createTrackbar('ValMin', 'Test', 50, 255, vMinCallback) 
cv2.createTrackbar('ValMax', 'Test', 200, 255, vMaxCallback) 

#frame_seq = 27950
#end_frame = 28250
frame_seq = 0
end_frame = 35000

cap = cv2.VideoCapture('WiiPlay.mp4')
if not cap.isOpened():
    raise IOError("Cannot open the video file")    
cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)

while True:
    ret, frame = cap.read()
    if ret == False:
        break
        
    cur_frame = cv2.resize(frame, None, fx=0.35, fy=0.35, interpolation=cv2.INTER_AREA)   
    cv2.imshow('Input', cur_frame)
    
    hsv_frame = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2HSV)
    
    HueMin = cv2.getTrackbarPos('HueMin', 'Test')
    HueMax = cv2.getTrackbarPos('HueMax', 'Test')
    SatMin = cv2.getTrackbarPos('SatMin', 'Test')
    SatMax = cv2.getTrackbarPos('SatMax', 'Test')
    ValMin = cv2.getTrackbarPos('ValMin', 'Test')
    ValMax = cv2.getTrackbarPos('ValMax', 'Test')
    
    lower = np.array([HueMin, SatMin, ValMin])
    upper = np.array([HueMax, SatMax, ValMax])
    mask = cv2.inRange(hsv_frame, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.erode(mask, kernel)
    mask = cv2.dilate(mask, kernel)
    cv2.imshow('Test', mask)
    
    lower_blue = np.array([100, 150, 46])
    upper_blue = np.array([124, 255, 255])
    blue_mask = cv2.inRange(hsv_frame, lower_blue, upper_blue)
    cv2.imshow('Cursor', blue_mask)
    
    lower_yellow = np.array([21, 203, 160])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv_frame, lower_yellow, upper_yellow)
    cv2.imshow('Timer', yellow_mask)
    
    lower_skin = np.array([7, 28, 50])
    upper_skin = np.array([20, 255, 255])
    skin_mask = cv2.inRange(hsv_frame, lower_skin, upper_skin)
    kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    kernel_2 = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
    skin_mask = cv2.erode(skin_mask, kernel_1)
    skin_mask = cv2.dilate(skin_mask, kernel_1)
    skin_mask = cv2.dilate(skin_mask, kernel_2)
    skin_mask = cv2.erode(skin_mask, kernel_2)
    cv2.imshow('Skin', skin_mask)

    colors = [
        (255, 0, 0),    
        (0, 255, 0),    
        (0, 0, 255),    
        (255, 255, 0),  
        (255, 0, 255),  
        (0, 255, 255),  
        (128, 0, 0),    
        (0, 128, 0),    
        (0, 0, 128),    
        (128, 128, 0),  
        (128, 0, 128),  
        (0, 128, 128),  
        (64, 0, 0),     
        (0, 64, 0),     
        (0, 0, 64),     
        (64, 64, 0),    
        (64, 0, 64),    
        (0, 64, 64),    
        (255, 128, 0),  
    ]
    
    num_labels, labels = cv2.connectedComponents(skin_mask)
    colored_skin_mask = np.zeros((skin_mask.shape[0], skin_mask.shape[1], 3), dtype=np.uint8)
    for label in range(1, min(num_labels, len(colors))):
        color = colors[label - 1]
        region_mask = np.uint8(labels == label) * 255
        colored_skin_mask[region_mask != 0] = color

    cv2.imshow('Count and Display', colored_skin_mask)
    
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
        break
    c = cv2.waitKey(1)
    if c == 27:
        break
    
cap.release()
cv2.destroyAllWindows()    
    